# Transformer for Java AST Processing

In [ ]:
# make sure we're in the right directory
%cd OpenNMT-py/

The files weren't encoded properly and so we have to convert them again...

In [19]:
%cd data/funcom_ast/

/home/jupyter/OpenNMT-py/data/funcom_ast


In [20]:
# rename files to something else
!mv test/comments.test test/comments2.test
!mv train/comments.train train/comments2.train
!mv valid/comments.valid valid/comments2.valid

In [21]:
# convert files to utf-8 with the name they're supposed to have
!iconv test/comments2.test -f utf-16 -t utf-8 -o test/comments.test
!iconv train/comments2.train -f utf-16 -t utf-8 -o train/comments.train
!iconv valid/comments2.valid -f utf-16 -t utf-8 -o valid/comments.valid

In [22]:
# remove the non-utf-7 files with the bobo name
!sudo rm test/comments2.test    
!sudo rm train/comments2.train  
!sudo rm valid/comments2.valid  

In [23]:
%cd ../..

/home/jupyter/OpenNMT-py


In [24]:
!python preprocess.py \
    -train_src data/funcom_ast/train/functions.train \
    -train_tgt data/funcom_ast/train/comments.train \
    -valid_src data/funcom_ast/valid/functions.valid \
    -valid_tgt data/funcom_ast/valid/comments.valid \
    -shard_size 100000 \
    -save_data data/funcom_ast/_prepped/prepped

[2019-06-07 10:30:52,247 INFO] Extracting features...
[2019-06-07 10:30:52,247 INFO]  * number of source features: 0.
[2019-06-07 10:30:52,247 INFO]  * number of target features: 0.
[2019-06-07 10:30:52,247 INFO] Building `Fields` object...
[2019-06-07 10:30:52,247 INFO] Building & saving training data...
[2019-06-07 10:30:52,247 INFO] Reading source and target files: data/funcom_ast/train/functions.train data/funcom_ast/train/comments.train.
[2019-06-07 10:30:52,527 INFO] Building shard 0.
[2019-06-07 10:30:58,562 INFO]  * saving 0th train data shard to data/funcom_ast/_prepped/prepped.train.0.pt.
[2019-06-07 10:31:01,547 INFO] Building shard 1.
[2019-06-07 10:31:07,562 INFO]  * saving 1th train data shard to data/funcom_ast/_prepped/prepped.train.1.pt.
[2019-06-07 10:31:10,657 INFO] Building shard 2.
[2019-06-07 10:31:16,791 INFO]  * saving 2th train data shard to data/funcom_ast/_prepped/prepped.train.2.pt.
[2019-06-07 10:31:19,770 INFO] Building shard 3.
[2019-06-07 10:31:25,993 IN

In [26]:
!python train.py \
    -data data/funcom_ast/_prepped/prepped \
    -save_model model/funcom_ast06082019 \
    -keep_checkpoint 1 \
    -layers 6 \
    -rnn_size 512 \
    -word_vec_size 512 \
    -transformer_ff 2048 \
    -heads 8 \
    -encoder_type transformer \
    -decoder_type transformer \
    -position_encoding \
    -train_steps 200000  \
    -max_generator_batches 0 \
    -dropout 0.1 \
    -batch_size 40 \
    -batch_type tokens \
    -normalization tokens \
    -accum_count 2 \
    -optim adam \
    -adam_beta2 0.998 \
    -decay_method noam \
    -warmup_steps 8000 \
    -learning_rate 0.2 \
    -max_grad_norm 0 \
    -param_init 0  \
    -param_init_glorot \
    -label_smoothing 0.1 \
    -valid_steps 200001 \
    -save_checkpoint_steps 10000 \
    -world_size 1 \
    -gpu_ranks 0

[2019-06-07 11:39:58,420 INFO]  * src vocab size = 50002
[2019-06-07 11:39:58,420 INFO]  * tgt vocab size = 50004
[2019-06-07 11:39:58,420 INFO] Building model...
[2019-06-07 11:40:03,466 INFO] NMTModel(
  (encoder): TransformerEncoder(
    (embeddings): Embeddings(
      (make_embedding): Sequential(
        (emb_luts): Elementwise(
          (0): Embedding(50002, 512, padding_idx=1)
        )
        (pe): PositionalEncoding(
          (dropout): Dropout(p=0.1)
        )
      )
    )
    (transformer): ModuleList(
      (0): TransformerEncoderLayer(
        (self_attn): MultiHeadedAttention(
          (linear_keys): Linear(in_features=512, out_features=512, bias=True)
          (linear_values): Linear(in_features=512, out_features=512, bias=True)
          (linear_query): Linear(in_features=512, out_features=512, bias=True)
          (softmax): Softmax()
          (dropout): Dropout(p=0.1)
          (final_linear): Linear(in_features=512, out_features=512, bias=True)
        )
     

In [29]:
!python translate.py \
    -model model/funcom_ast06082019_step_200000.pt \
    -src data/funcom_ast/test/functions.test \
    -tgt data/funcom_ast/test/comments.test \
    -output pred.txt \
    -replace_unk \
    -report_time 

[2019-06-08 07:21:06,330 INFO] Translating shard 0.
PRED AVG SCORE: -1.1883, PRED PPL: 3.2814
GOLD AVG SCORE: -4.5490, GOLD PPL: 94.5402
Total translation time (s): 7480.608321
Average translation time (s): 0.748061
Tokens per second: 12.251143
[2019-06-08 09:25:47,411 INFO] Translating shard 1.
PRED AVG SCORE: -1.0434, PRED PPL: 2.8389
GOLD AVG SCORE: -4.3576, GOLD PPL: 78.0684
Total translation time (s): 7065.386111
Average translation time (s): 0.706539
Tokens per second: 13.417384
[2019-06-08 11:23:33,689 INFO] Translating shard 2.
PRED AVG SCORE: -1.2222, PRED PPL: 3.3947
GOLD AVG SCORE: -4.4564, GOLD PPL: 86.1808
Total translation time (s): 7849.549870
Average translation time (s): 0.784955
Tokens per second: 11.483206
[2019-06-08 13:34:24,245 INFO] Translating shard 3.
PRED AVG SCORE: -1.2154, PRED PPL: 3.3717
GOLD AVG SCORE: -4.6103, GOLD PPL: 100.5163
Total translation time (s): 8107.848790
Average translation time (s): 0.810785
Tokens per second: 11.153267
[2019-06-08 15:49:3